# Feature Pipeline
Download the dataset, transform it and save it in Hopsworks

In [ ]:
!add-apt-repository -y ppa:jonathonf/ffmpeg-4
!apt update
!apt install -y ffmpeg

In [ ]:
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "sv-SE", split="train", use_auth_token=True)
common_voice["val"] = load_dataset("mozilla-foundation/common_voice_11_0", "sv-SE", split="validation", use_auth_token=True)
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "sv-SE", split="test", use_auth_token=True)

print(common_voice)

In [ ]:
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])

print(common_voice)

In [ ]:
import random
common_voice["train"] = common_voice["train"].select(random.choices(range(len(common_voice["train"])), k=2000))
common_voice["val"] = common_voice["val"].select(random.choices(range(len(common_voice["val"])), k=500))
common_voice["test"] = common_voice["test"].select(random.choices(range(len(common_voice["test"])), k=500))

In [ ]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

In [ ]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Swedish", task="transcribe")

In [ ]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Swedish", task="transcribe")

In [ ]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

In [ ]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array 
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids 
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

In [ ]:
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=2)

In [ ]:
common_voice.save_to_disk("common_voice")

In [ ]:
!pip install hopsworks

In [ ]:
import hopsworks
project = hopsworks.login()

In [ ]:
dataset_api = project.get_dataset_api()

In [ ]:
!sudo apt install zip

In [ ]:
!zip -r all_data.zip common_voice/

In [ ]:
!du -sh ./all_data.zip

In [ ]:
uploaded_file_path = dataset_api.upload(local_path = "./all_data.zip", upload_path = "Resources/Translator", overwrite=True)